In [ ]:
import numpy as np
from scipy.stats import norm

from ipywidgets import FloatSlider, HBox, VBox
import bqplot.pyplot as plt
x = np.linspace(-10, 10, 200)
y = norm.pdf(x)

# plot the gaussian density
title_tmpl = 'Gaussian Density (mu = {} and sigma = {})'
pdf_fig = plt.figure(title=title_tmpl.format(0, 1))
pdf_line = plt.plot(x, y, 'm', stroke_width=3)

# use two sliders to represent mu and sigma
mu_slider = FloatSlider(description='mu', value=0, min=-5, max=5, step=.1)
sigma_slider = FloatSlider(description='sigma', value=1, min=0.1, max=5, step=.1)

slider_layout = HBox([mu_slider, sigma_slider])

def update_density(change):
    new_mu = mu_slider.value
    new_sigma = sigma_slider.value
    # update the y attribute of the plot with the new pdf
    # computed using new mu and sigma values
    pdf_line.y = norm.pdf(x, new_mu, new_sigma)
    
    # also update the fig title
    pdf_fig.title = title_tmpl.format(new_mu, new_sigma)

# register the above callback with the 'value' trait of the sliders
for slider in [mu_slider, sigma_slider]:
    slider.observe(update_density, 'value')
    
# now put all the widgets together into a simple dashboard
# the plot should update now when the slider values are updated!
final_layout = VBox([pdf_fig, slider_layout])
final_layout

In [ ]:
import ROOT

In [ ]:
ROOT.gSystem.Load("libQuantLib.so")

In [ ]:
%%cpp
#include <ql/quantlib.hpp>
#include <sstream>

In [ ]:
%%cpp
using namespace QuantLib;

In [ ]:
%%cpp 
class Pricer {
public:    
    Pricer() {};
std::string output() {
    std::stringstream s;
// date set up
Calendar calendar = TARGET();
Date todaysDate(27, Jan, 2011);
Date settlementDate(27, Jan, 2011);
Settings::instance().evaluationDate() = todaysDate;

// option parameters
Option::Type type(Option::Call);
Real stock = 47;
Real strike = 40;
Spread dividendYield = 0.00;
Rate riskFreeRate = 0.05;
Volatility volatility = 0.20;
Date maturity(27, May, 2011);
DayCounter dayCounter = Actual365Fixed();

std::shared_ptr<Exercise> europeanExercise(new EuropeanExercise(maturity));

Handle<Quote> underlyingH(std::shared_ptr<Quote>(new SimpleQuote(stock)));

// bootstrap the yield/dividend/vol curves
Handle<YieldTermStructure> flatTermStructure(ext::shared_ptr<YieldTermStructure>(
	new FlatForward(
	settlementDate,
	riskFreeRate,
	dayCounter)));

Handle<YieldTermStructure> flatDividendTS(ext::shared_ptr<YieldTermStructure>(
	new FlatForward(settlementDate,
	dividendYield,
	dayCounter)));

Handle<BlackVolTermStructure> flatVolTS(ext::shared_ptr<BlackVolTermStructure>(
	new BlackConstantVol(
	settlementDate,
	calendar,
	volatility,
	dayCounter)));

ext::shared_ptr<StrikedTypePayoff> payoff(
	new PlainVanillaPayoff(
	type,
	strike));

ext::shared_ptr<BlackScholesMertonProcess> bsmProcess(
	new BlackScholesMertonProcess(
	underlyingH,
	flatDividendTS,
	flatTermStructure,
	flatVolTS));

// our option is European-style
VanillaOption europeanOption(
	payoff,
	europeanExercise);

// computing the option price with the analytic Black-Scholes formulae
europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
	new AnalyticEuropeanEngine(
	bsmProcess)));

// outputting
s << "Option type = " << type << std::endl;
s << "Maturity = " << maturity << std::endl;
s << "Stock price = " << stock << std::endl;
s << "Strike = " << strike << std::endl;
s << "Risk-free interest rate = " << io::rate(riskFreeRate) << std::endl;
s << "Dividend yield = " << io::rate(dividendYield) << std::endl;
s << "Volatility = " << io::volatility(volatility) << std::endl << std::endl;
s <<"European Option value = " << europeanOption.NPV() << std::endl;
    return s.str();
}
};

In [ ]:
b = ROOT.Pricer()

In [ ]:
c = b.output()

In [ ]:
c

In [ ]:
import ipywidgets
foodate = ipywidgets.widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)

button = ipywidgets.Button(description="Click Me!")
output = ipywidgets.Output();

def on_button_clicked(button):
    with output:
        print(b.output())

button.on_click(on_button_clicked)

In [ ]:
display(foodate, button, output)

In [ ]:
foodate.value